In [1]:
!pip install setfit==1.0.3 transformers==4.37.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 4.6 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_6

# Intent Detection

In [2]:
from datasets import DatasetDict, Dataset
from setfit import SetFitModel, Trainer, TrainingArguments
import random
import shutil
import pandas as pd

In [3]:
path = "/content/football_sft_model"

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Data Preparation

In [4]:
data = {
    'greet-hi': [
                            'Hi',
                            'Hoooo',
                            'Hello buddy',
                            'Salut',
                            'Hey',
                            'Good morning',
                            'Bonsoir',
                            'Salut mon ami'
                        ],
    'greet-who_are_you': [
                        'qui êtes-vous',
                        'who are you',
                        'how can you help me',
                        "comment pouvez vous m'aider",
                        'why do I need you',
                        "pourquoi j'ai besoin de toi",
                        'help please',
                        "Aidez-moi, s'il vous plaît",
                        'I can not understand you',
                        'je ne te comprends pas'
                    ],
    'greet-good_bye': ['good bye',
                    'au revoir',
                    'See you later',
                    'A plus tard',
                    'stop',
                    'arrêter',
                    'exit',
                    'sortir'],
    'matches-team_next_match': [
                        'when will be the matches of Liverpool',
                        'quand auront lieu les matchs de Liverpool',
                        'when will be the matches of Aston Villa',
                        'quand auront lieu les matchs de Aston Villa',
                        'man city matches',
                        'Arsenal matches',
                        'Arsenal matchs',
                        'man city matchs',
                        'what the calendar of chelsea matches',
                        'à quoi correspond le calendrier de chelsea',
                        'Will Liverpool play any matches soon',
                        'Est-ce que Liverpool jouera bientôt des matchs',
                        'Will Liverpool play any matches next days',
                        'Est-ce que Liverpool jouera des matchs les jours prochains',
                        'upcoming games for Manchester United',
                        'prochains matchs de Tottenham Hotspur',
                        'Barcelona match schedule',
                        'Real Madrid fixtures',
                        'when are the matches of Bayern Munich',
                        'calendrier des matchs pour Paris Saint-Germain',
                        'any upcoming fixtures for Juventus',
                        'próximos partidos de Atletico Madrid',
                        'which teams will AC Milan face next',
                        'quels sont les prochains matchs de Borussia Dortmund'
                    ],
    'matches-match_time': [
                            'when liverpool will play with man city',
                            'quand liverpool jouera avec man city',
                            'crystal place vs chelsea',
                            'Bournemouth vs Brentford',
                            'Bournemouth vs Brentford',
                            'Norwich City vs Newcastle United',
                            'Southampton vs West Ham United',
                            'tell me the time of Everton vs Leicester City',
                            'tell me the time of Wolverhampton vs Chelsea',
                            'will Manchester United play with chelsea',
                            'will Brentford play with Bournemouth',
                            'Dis-moi le temps de Everton contre Leicester City',
                            'dis-moi le temps de Wolverhampton vs Chelsea',
                            'est-ce que Manchester United jouera avec chelsea',
                            'est-ce que Brentford jouera avec Bournemouth'
                        ],
    'matches-match_result': [
                            'what is the score of Brentford match',
                            'what is the score of Wolverhampton match',
                            'score of Everton match',
                            'who won in Norwich City vs West Ham United match',
                            'who won in Bournemouth vs West Ham United match',
                            'who won in Liverpool vs Newcastle United match',
                            'did Liverpool defeted man city',
                            'Liverpool and West Ham result',
                            'Chelsea and Norwich final result',
                            'quel est le score du match Brentford',
                            'quel est le score du match Wolverhampton',
                            'score du match Everton',
                            'qui a gagné le match Norwich City contre Norwich City',
                            'qui a gagné le match Bournemouth contre West Ham United',
                            'qui a gagné le match Liverpool contre Newcastle United',
                            'est-ce que Liverpool a vaincu man city',
                            'Résultat de Liverpool et West Ham',
                            'Résultat final de Chelsea et Norwich'
                        ]
}

In [5]:
train_data, dev_data = [], []
intents = list(data.keys())
idx = 0

for intent, prompts in data.items():

    random.Random(1).shuffle(prompts)
    intent_id = intents.index(intent)

    train_prompts = prompts[:-3]
    dev_prompts = prompts[-3:]

    for prompt in train_prompts:
        train_data.append({
            "idx": idx,
            "prompt": prompt,
            "intent": intent_id
        })
        idx+=1

    for prompt in dev_prompts:
        dev_data.append({
            "idx": idx,
            "prompt": prompt,
            "intent": intent_id
        })
        idx+=1

random.Random(1).shuffle(train_data)
random.Random(1).shuffle(dev_data)

# Create full dataset dictionary
dataset = DatasetDict({
    "train": Dataset.from_pandas(pd.DataFrame(train_data)),
    "dev": Dataset.from_pandas(pd.DataFrame(dev_data))
})

### Train

In [6]:
# Embedding Model
embed_model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    labels = intents
)

config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.12k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [7]:
# Training parameters

args = TrainingArguments(
    batch_size = 4,
    num_epochs = 4,
    logging_steps = 10,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True
)

trainer = Trainer(
    model = embed_model,
    args = args,
    train_dataset = dataset["train"],
    eval_dataset = dataset["dev"],
    metric = "accuracy",
    column_mapping = {"prompt": "text", "intent": "label"}
)

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/65 [00:00<?, ? examples/s]

In [8]:
trainer.train()

***** Running training *****
  Num unique pairs = 3316
  Batch size = 4
  Num epochs = 4
  Total optimization steps = 3316


Epoch,Training Loss,Validation Loss,Embedding Loss,Rate
1,No log,No log,0.009300,0.000017
2,No log,No log,0.007100,0.000011
3,No log,No log,0.007100,0.000006
4,No log,No log,0.006400,0.000000


  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

  0%|          | 0/68 [00:00<?, ?it/s]

Loading best SentenceTransformer model from step 3316.


### Evaluation

In [9]:
metrics = trainer.evaluate(dataset["dev"])
print(metrics)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
Applying column mapping to the evaluation dataset
***** Running evaluation *****


{'accuracy': 1.0}


### Saving Locally

In [10]:
# save and zip locally
trainer.model.save_pretrained(path)
shutil.make_archive(path, "zip", path)

'/content/football_sft_model.zip'

In [ ]:
# push to hf
trainer.model.push_to_hub(
    repo_id = "botpress_football_sft_model",
    private = True,
)

model_head.pkl:   0%|          | 0.00/19.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/471M [00:00<?, ?B/s]

unigram.json:   0%|          | 0.00/14.8M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/MuhammadHelmy/botpress_football_sft_model/commit/89d443ab6d167a29fbf942910ab932d7b9e86f28', commit_message='Push model using huggingface_hub.', commit_description='', oid='89d443ab6d167a29fbf942910ab932d7b9e86f28', pr_url=None, pr_revision=None, pr_num=None)

### Load from local

In [21]:
intent_model = SetFitModel.from_pretrained("MuhammadHelmy/botpress_football_sft_model")

### Testing Inferencing

In [22]:
preds = intent_model.predict([
    "Hi everybody!",
    "When Liverpool will play?",
    "من الذي فاز في مباراة الأهلي والزمالك؟"
])

In [23]:
preds

['greet-hi', 'matches-match_time', 'matches-match_result']

In [24]:
probs = intent_model.predict_proba([
    "Hi everybody!",
    "When Liverpool will play?",
    "من الذي فاز في مباراة الأهلي والزمالك؟"
]).tolist()

In [25]:
probs

[[0.9393362511344264,
  0.0252376577175864,
  0.008357781965532702,
  0.008715160074826986,
  0.00805322318198944,
  0.01029992592563807],
 [0.01075641756463522,
  0.009948115822205611,
  0.0096619337604315,
  0.04186757393320049,
  0.9167862472702646,
  0.010979711649262607],
 [0.0014734082482331652,
  0.0020127048162162124,
  0.001548008371522715,
  0.0014040633566333015,
  0.0011047533259049493,
  0.9924570618814896]]

# Named Entity Recognition

In [ ]:
!pip install gliner==0.2.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.7 MB/s eta 0:00:00
  Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 105.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 97.0 MB/s eta 0:00:00
Using cached tokenizers-0.19.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.6 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 9.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: transformers 4.37.0
    Uninstalling transformers-4.37.0:
      Successfully unin

In [ ]:
from gliner import GLiNER
import pprint

In [ ]:
ner_model = GLiNER.from_pretrained("urchade/gliner_multi-v2.1")

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

gliner_config.json:   0%|          | 0.00/477 [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.77k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.16G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/579 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:562: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
samples = [
    "who won between Manchester City and Arsenal?",
    "كيف انتهت مباراة نورويتش سيتي وتشيلسي"
]

labels = ["team_name"]

for text in samples:
    entities = ner_model.predict_entities(text, labels)
    pprint.pprint(entities)
    print("-------------------------")

[{'end': 31,
  'label': 'team_name',
  'score': 0.7424863576889038,
  'start': 16,
  'text': 'Manchester City'},
 {'end': 43,
  'label': 'team_name',
  'score': 0.517137885093689,
  'start': 36,
  'text': 'Arsenal'}]
-------------------------
[{'end': 29,
  'label': 'team_name',
  'score': 0.7897310256958008,
  'start': 17,
  'text': 'نورويتش سيتي'}]
-------------------------


In [ ]:
txt = "Mohamed Mohamed Mohamed Aboutrika is an Egyptian retired professional footballer who played as an attacking midfielder and a forward for Al-ahly. He was voted second place in the African Footballer of the Year award in 2008, and was one of five nominees for the 2006 award, and one of the ten nominated for the 2013 award.".strip()
labels = ["team_name", "person_name", "event"]

entities = ner_model.predict_entities(txt, labels)
pprint.pprint(entities)

[{'end': 33,
  'label': 'person_name',
  'score': 0.8084983229637146,
  'start': 0,
  'text': 'Mohamed Mohamed Mohamed Aboutrika'},
 {'end': 144,
  'label': 'team_name',
  'score': 0.8981382846832275,
  'start': 137,
  'text': 'Al-ahly'},
 {'end': 215,
  'label': 'event',
  'score': 0.8969795107841492,
  'start': 179,
  'text': 'African Footballer of the Year award'},
 {'end': 272,
  'label': 'event',
  'score': 0.8427568078041077,
  'start': 262,
  'text': '2006 award'},
 {'end': 321,
  'label': 'event',
  'score': 0.8318695425987244,
  'start': 311,
  'text': '2013 award'}]
